Notebook for creating Combined Trends over Time Graph

In [1]:
import altair as alt
import pandas as pd
import streamlit as st
from base_script import return_merged_df, get_connection

Import merged df

In [2]:
conn = get_connection()
merged_df = return_merged_df()
merged_df.sample(5)

,recording_id,plant_id,last_watered,soil_moisture,temperature,recording_taken
2570,7785,26,2024-11-28 14:23:18,21.040682,14.142354,2024-11-29 14:13:46
5566,4789,11,2024-11-28 13:37:24,20.553158,9.444513,2024-11-29 13:11:33
4612,5743,1,2024-11-28 13:54:32,16.325087,12.037304,2024-11-29 13:31:25
9210,1156,20,2024-11-28 14:49:34,25.465661,8.558238,2024-11-29 12:00:22
4561,5794,5,2024-11-28 14:56:18,23.018509,11.200284,2024-11-29 13:32:27


In [3]:
alt.data_transformers.disable_max_rows()
merged_df = merged_df.drop(columns=['recording_id', 'plant_id', 'last_watered'])
merged_df.sample(5)

merged_df.set_index('recording_taken', inplace=True)
grouped_df = merged_df.resample('30min').mean()
grouped_df = grouped_df.interpolate(method='linear').reset_index()

# merged_df['recording_taken'] = merged_df['recording_taken'].dt.floor('H')
# merged_df = merged_df.groupby('recording_taken').mean().reset_index()

In [4]:
long_df = grouped_df.melt(
    id_vars='recording_taken',
    value_vars=['soil_moisture', 'temperature'],
    var_name='Measurement',
    value_name='Value'
)

# Create the Altair line chart
chart = alt.Chart(long_df).mark_line().encode(
    x=alt.X('recording_taken:T', title='Time'),
    y=alt.Y('Value:Q', title='Value'),
    color='Measurement:N',
    tooltip=['recording_taken:T', 'Measurement:N', 'Value:Q']
).properties(
    title='30-Minute Average Soil Moisture and Temperature over Time',
    width=800,
    height=400
)

chart.show()

alt.Chart(...)